<a href="https://colab.research.google.com/github/Tirta2241720045/Machine_Learning/blob/main/Minggu11/2241720045_TirtaNBP_Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nama: Tirta Nurrochman Bintang Prawira
# NIM: 2241720045
# Kelas: TI-3A
# No Absen: 27

# Tugas
- Buatlah model CNN berdasarkan dataset citra LFW.